<a href="https://colab.research.google.com/github/FernandaSzAdr/CNN_cromossome/blob/master/TesteRede5_Colorida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install imutils

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers.core import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score

from skimage import transform

import cv2
import csv
import imutils
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import threading

Using TensorFlow backend.


## Funções

In [0]:
def converte(regiao):
   regiao = np.array(regiao).astype('float32')/255
   regiao = transform.resize(regiao, (120, 120, 3))
   regiao = np.expand_dims(regiao, axis=0)
   return regiao


[NMS Object Detection](https://www.pyimagesearch.com/2014/11/17/non-maximum-suppression-object-detection-python)

In [0]:
def non_max_suppression_slow(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []

	# initialize the list of picked indexes
	pick = []

	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]

	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)

	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list, add the index
		# value to the list of picked indexes, then initialize
		# the suppression list (i.e. indexes that will be deleted)
		# using the last index
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		suppress = [last]

		# loop over all indexes in the indexes list
		for pos in range(0, last):
			# grab the current index
			j = idxs[pos]

			# find the largest (x, y) coordinates for the start of
			# the bounding box and the smallest (x, y) coordinates
			# for the end of the bounding box
			xx1 = max(x1[i], x1[j])
			yy1 = max(y1[i], y1[j])
			xx2 = min(x2[i], x2[j])
			yy2 = min(y2[i], y2[j])

			# compute the width and height of the bounding box
			w = max(0, xx2 - xx1 + 1)
			h = max(0, yy2 - yy1 + 1)

			# compute the ratio of overlap between the computed
			# bounding box and the bounding box in the area list
			overlap = float(w * h) / area[j]

			# if there is sufficient overlap, suppress the
			# current bounding box
			if overlap > overlapThresh:
				suppress.append(pos)

		# delete all indexes from the index list that are in the
		# suppression list
		idxs = np.delete(idxs, suppress)

	# return only the bounding boxes that were picked
	return boxes[pick]

 [Sliding Windows](https://www.pyimagesearch.com/2015/03/23/sliding-windows-for-object-detection-with-python-and-opencv/)





In [0]:
def sliding_window(image, stepSize, windowSize):
	# slide a window across the image
	for y in range(0, image.shape[0], stepSize):
		for x in range(0, image.shape[1], stepSize):
			# yield the current window
			yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [0]:
def regioes_cromossomos(image):
  f = open('drive/My Drive/Base_CNN/Binarizado2/posicoes_segmentada.csv', 'r')
  boxesOriginal = []
  
  try:
    leitor = csv.reader(f)
    leitor = list(leitor)
    leitor.pop(0)

    for linha in leitor:
      if len(linha) > 0:
        pick = []
        # Para cada região dessa imagem:
        if linha[0] == image and linha[1] == 'C':
          x = int(linha[3])
          y = int(linha[4])
          w = int(linha[5])
          h = int(linha[6])
          pick.extend([x, y, x+w, y+h])        
          
          boxesOriginal.append(pick)

  finally:
    f.close()
    
  return boxesOriginal

[Intersection Over Union (IoU)](https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/)

[More about mAP](https://www.pyimagesearch.com/2018/05/14/a-gentle-guide-to-deep-learning-object-detection/)

[The confusing metrics of AP and mAP for object detection](https://mc.ai/the-confusing-metrics-of-ap-and-map-for-object-detection/)

In [0]:
def calcula_iou(boxesPredicao, boxesOriginal, local_arquivos, local, img, image, taxa_iou):
  
  with open(local_arquivos + '/precisao_images.csv', 'a', newline='') as csv_file:
    fieldnames = ['image', 'precision', 'taxa_iou']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)    
  
    image_clone = image.copy()

    TP = 0
    FP = 0

    for (startX_pred, startY_pred, endX_pred, endY_pred) in boxesPredicao:
      for (startX_ori, startY_ori, endX_ori, endY_ori) in boxesOriginal:
        xA = max(startX_pred, startX_ori)
        yA = max(startY_pred, startY_ori)
        xB = min(endX_pred, endX_ori)
        yB = min(endY_pred, endY_ori)

        # compute the area of intersection rectangle
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

        # compute the area of both the prediction and ground-truth
        # rectangles
        boxAArea = (endX_pred - startX_pred + 1) * (endY_pred - startY_pred + 1)
        boxBArea = (endX_ori - startX_ori + 1) * (endY_ori - startY_ori + 1)

        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)

        if iou > taxa_iou:
          cv2.rectangle(image,(startX_pred, startY_pred),(endX_pred, endY_pred),
                        (255, 0, 0), 1)
          cv2.rectangle(image,(startX_ori, startY_ori),(endX_ori, endY_ori),
                        (0, 255, 0), 1)

          cv2.rectangle(image_clone,(startX_pred, startY_pred),(endX_pred, endY_pred),
                        (255, 0, 0), 1)
          cv2.putText(image_clone, "IoU: {:.4f}".format(iou), (startX_pred+15,
                                                         startY_pred),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

          TP += 1
        else:
          FP += 1
    if TP+FP == 0:
      precision = 0
    else:
      precision = TP / (TP+FP)

    cv2.imwrite(local + '/iou' + str(taxa_iou) + '_img' + img, image)
    cv2.imwrite(local + '/iou_taxas' + str(taxa_iou) + '_img' + img, image_clone)
    
    writer.writeheader()
    writer.writerow({
      'image': img,
      'precision': precision,
      'taxa_iou': taxa_iou
    })

  return precision

In [0]:
def ler_images_teste(architecture, model):
  test_path = 'drive/My Drive/Base_CNN/Colorido2/Test'
  images_test = os.listdir(test_path)
  
  mAP_values_total = []
  
#   for bSize in range(10, 21, 5):
  for bSize in range(20, 21, 5):
    for win in range(20, 41, 5):
      print('bSize {} -  win {}'.format(bSize, win))
      (winW, winH) = (win, win)
      
      local_mAP = 'drive/My Drive/Resultado/Colorido/{}'.format(architecture)
      local_arquivos = local_mAP + '/Janela{}-Batch{}'.format(winH, bSize)
      local = local_arquivos + '/Resultado_Image'
      if os.path.isdir(local) == False:
        os.makedirs(local)
        print('Criou-se a pasta {}'.format(local))

      mAP_values = []

      with open(local_mAP + '/average_precision.csv', 'a', newline='') as csv_file:
        fieldnames = ['average_precision', 'taxa_iou', 'precision', 'janela', 'batch']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        for taxa_iou in range(4, 9):
          local_iou = local + '/IOU{}'.format(taxa_iou)
          if os.path.isdir(local_iou) == False:
            os.mkdir(local_iou)
            print('Criou-se a pasta {}'.format(local_iou))

          print("IoU {}".format(taxa_iou))
          taxa_iou = taxa_iou/10
          precisao_total = 0

          for img in images_test:
            print("Run image {}".format(img))
            # Pega cada imagem separadamente do folder de teste
            image = cv2.imread(test_path + '/' + img)
            image = imutils.resize(image, width=1000)
            image_after = image.copy()

            boxesOriginal = regioes_cromossomos(img)

            image_sem_formato = img.split('.tif')[0]
            img = image_sem_formato + '.png'

            boudingBoxes = []

            # sliding window
            for (x, y, window) in sliding_window(image, stepSize=bSize, windowSize=(winW, winH)):
              # if the window does not meet our desired window size, ignore it
              if window.shape[0] != winH or window.shape[1] != winW:
                continue

              regiao = converte(window)

              # Coloca a região na função e obtem o resultado
              predicao = model.predict(regiao, batch_size=bSize, verbose=0)

              # Se é um cromossomo:
              if predicao[0][1] > predicao[0][0]:
                endX = x + winW
                endY = y + winH

                boudingBox = (x, y, endX, endY)
                boudingBoxes.append(boudingBox)

            boxesPredicao = np.array(boudingBoxes)

            precisao_total += calcula_iou(boxesPredicao, boxesOriginal, local_arquivos,
                                                    local_iou, img, image, taxa_iou)

          average_precision = precisao_total / len(images_test)
          mAP_values.append(average_precision)

        writer.writeheader()
        writer.writerow({
          'average_precision': average_precision,
          'precision': precisao_total,
          'janela': winH,
          'batch': bSize
        })

      mAP_values_total.append({
          'win': win,
          'bSize': bSize,
          'mAP_values': mAP_values,
          'media': np.mean(mAP_values)}) 
  
  return mAP_values_total



---



# Implementação



*   Batch size = 32
*   Image size = 120x120



## Variaveis globais



*   Batch size = 32
*   Image size = 120x120



In [0]:
train_path = 'drive/My Drive/Base_CNN/Colorido2/Train'
test_path = 'drive/My Drive/Base_CNN/Colorido2/Test'
validation_path = 'drive/My Drive/Base_CNN/Colorido2/Validation'

In [0]:
results = {}

In [0]:
bSize = 32

In [0]:
datagen = ImageDataGenerator(rescale=1/255.)

In [14]:
train_batches = datagen.flow_from_directory(train_path, target_size=(120, 120), 
                                            classes=['NC', 'C'], 
                                            batch_size=bSize)
validation_batches = datagen.flow_from_directory(validation_path, 
                                                 target_size=(120, 120), 
                                                 classes=['NC', 'C'], 
                                                 batch_size=bSize)
stepsPerEpoche = len(train_batches)/bSize
validationSteps = len(validation_batches)/bSize

Found 4369 images belonging to 2 classes.
Found 991 images belonging to 2 classes.


In [0]:
train_batches.class_indices

{'C': 1, 'NC': 0}



---



## 1 tentativa - 

**Descrição**


*   Modelo descrito no artigo "Crowdsourcing for Chromosome Segmentation and Deep Classification"



In [0]:
architecture = 'architecture_1'

### Modelo

In [0]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(120, 120, 3)),
    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(32, (3, 3), activation='relu'),
    Conv2D(32, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(16, (3, 3), activation='relu'),
    Conv2D(16, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(8, (3, 3), activation='relu'),
    Conv2D(8, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(1024, activation='sigmoid'),
    Dense(512, activation='sigmoid'),
   
    Dense(2, activation='softmax')
]
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_1/Janela20-Batch20/Resultado_Image/IOU5
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_1/Janela20-Batch20/Resultado_Image/IOU6
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Result

[{'bSize': 20,
  'mAP_values': [0.0, 0.0, 0.0, 0.0, 0.0],
  'media': 0.0,
  'win': 20},
 {'bSize': 20,
  'mAP_values': [0.0, 0.0, 0.0, 0.0, 0.0],
  'media': 0.0,
  'win': 25},
 {'bSize': 20,
  'mAP_values': [0.0, 0.0, 0.0, 0.0, 0.0],
  'media': 0.0,
  'win': 30},
 {'bSize': 20,
  'mAP_values': [0.0, 0.0, 0.0, 0.0, 0.0],
  'media': 0.0,
  'win': 35},
 {'bSize': 20,
  'mAP_values': [0.0, 0.0, 0.0, 0.0, 0.0],
  'media': 0.0,
  'win': 40}]

## 2 tentativa  - 

**Descrição**


*   Modelo descrito no artigo "Crowdsourcing for Chromosome Segmentation and Deep Classification"
*   Alteração nas duas ultimas camadas Dense





In [0]:
architecture = 'architecture_2'

### Modelo

In [0]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(120, 120, 3)),
    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(32, (3, 3), activation='relu'),
    Conv2D(32, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(16, (3, 3), activation='relu'),
    Conv2D(16, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
   
    Conv2D(8, (3, 3), activation='relu'),
    Conv2D(8, (3, 3), activation='relu'),
    Dropout(0.2),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
   
    Dense(2, activation='softmax')
]
)

In [0]:
model.compile(SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 7
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run 

[{'bSize': 20,
  'mAP_values': [0.004667257065534005,
   0.0019248788195350996,
   0.0006907514779407216,
   0.0001665918547648181,
   0.0],
  'media': 0.0014898958435549286,
  'win': 20},
 {'bSize': 20,
  'mAP_values': [0.006973692652409022,
   0.0034468407059735084,
   0.0011053370194927456,
   0.00031594829647716566,
   0.0],
  'media': 0.0023683637348704883,
  'win': 25},
 {'bSize': 20,
  'mAP_values': [0.00715606911490917,
   0.0031731782209042155,
   0.0014308854981042256,
   0.0003072052059424139,
   2.7745678655688768e-05],
  'media': 0.002419016743703143,
  'win': 30},
 {'bSize': 20,
  'mAP_values': [0.006592586587418231,
   0.0028470621531870082,
   0.0011353259643906287,
   0.0005152716467623483,
   0.00010101010101010101],
  'media': 0.0022382512905536637,
  'win': 35},
 {'bSize': 20,
  'mAP_values': [0.005324176059286188,
   0.0018370414171299458,
   0.0006516651254069354,
   0.00018357487922705316,
   0.0],
  'media': 0.0015992914962100243,
  'win': 40}]

## 3 tentativa - 

Modelo criado pelo **Lucas Valentim**

https://github.com/lucasvalentim/chromosome-recognizer/blob/master/chromosome_recognizer.ipynb

In [0]:
architecture = 'architecture_3'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid', kernel_regularizer=l2(1e-4)))

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_3/Janela20-Batch20/Resultado_Image
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_3/Janela20-Batch20/Resultado_Image/IOU4
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_3/Janela20-Batch20/Resultado_Image/IOU5
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_3/Janela20-Batch20/Resultado_Image/IOU6
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Result

[{'bSize': 20,
  'mAP_values': [0.0034685576968738543,
   0.0006931734297766963,
   0.00022215739019878375,
   4.005533518702973e-05,
   0.0],
  'media': 0.0008847887704072728,
  'win': 20},
 {'bSize': 20,
  'mAP_values': [0.0021382765651147845,
   0.0005892018751189338,
   0.00025114235148074364,
   0.00010582010582010582,
   0.0],
  'media': 0.0006168881795069136,
  'win': 25},
 {'bSize': 20,
  'mAP_values': [0.002678096305131676,
   0.001489136600584481,
   0.00042197576279405047,
   1.2493753123438281e-05,
   0.0],
  'media': 0.0009203404843267293,
  'win': 30},
 {'bSize': 20,
  'mAP_values': [0.0020156275376756443,
   0.0011505055471736885,
   0.0005826910146062614,
   3.4343917331724294e-05,
   0.0],
  'media': 0.0007566336033574636,
  'win': 35},
 {'bSize': 20,
  'mAP_values': [0.0031357264283426657,
   0.0004999685809143381,
   0.00028088605705492637,
   0.0,
   0.0],
  'media': 0.0007833162132623859,
  'win': 40}]

## 4 tentativa - 

Modelo adaptado do **Lucas Valentim**

*   Sem os **dropout**


In [0]:
architecture = 'architecture_4'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Dense(2, activation='sigmoid', kernel_regularizer=l2(1e-4)))

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_4/Janela20-Batch20/Resultado_Image
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_4/Janela20-Batch20/Resultado_Image/IOU4
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_4/Janela20-Batch20/Resultado_Image/IOU5
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_4/Janela20-Batch20/Resultado_Image/IOU6
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Result

## 5 tentativa - 

Modelo adaptado do **Lucas Valentim**

*   Sem os **dropout**
*   Função **SGD** para compilação


In [0]:
architecture = 'architecture_5'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid', kernel_regularizer=l2(1e-4)))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 7
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run 

[{'bSize': 20,
  'mAP_values': [0.002467298709919284,
   0.0007103723812651122,
   0.00019425932919454262,
   4.833574490300781e-05,
   0.0],
  'media': 0.0006840532330563894,
  'win': 20},
 {'bSize': 20,
  'mAP_values': [0.003417415069212983,
   0.0013139784033231654,
   0.00034121370676077603,
   0.00013531294086268012,
   0.0],
  'media': 0.001041584024031921,
  'win': 25},
 {'bSize': 20,
  'mAP_values': [0.0036546815100209503,
   0.0014511845204165973,
   0.0005525059769112683,
   0.00016859325042375932,
   7.919537499010058e-06],
  'media': 0.001166976959054317,
  'win': 30},
 {'bSize': 20,
  'mAP_values': [0.003516256878239314,
   0.0013201702960045046,
   0.00037717875515980853,
   8.881451459050311e-05,
   4.3572984749455335e-05],
  'media': 0.001069198685748717,
  'win': 35},
 {'bSize': 20,
  'mAP_values': [0.0029782535793999396,
   0.0008364692288146416,
   0.00033583763549848854,
   4.535147392290249e-05,
   0.0],
  'media': 0.0008391823835271945,
  'win': 40}]

## 6 tentativa - 

Modelo adaptado do **Lucas Valentim**

*   Sem os **dropout**
*   Função **SGD** para compilação
*   **BatchNormalization** antes da ultima camada Dense


In [0]:
architecture = 'architecture_6'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Dense(2, activation='sigmoid', kernel_regularizer=l2(1e-4)))

In [0]:
model.compile(SGD(lr=0.03, momentum=0.0, decay=0.0, nesterov=False), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_6/Janela20-Batch20/Resultado_Image
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_6/Janela20-Batch20/Resultado_Image/IOU4
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_6/Janela20-Batch20/Resultado_Image/IOU5
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_6/Janela20-Batch20/Resultado_Image/IOU6
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Result

[{'bSize': 20,
  'mAP_values': [0.001755885323279662,
   0.0006878536399278004,
   0.00026981737386122626,
   9.756776494198044e-05,
   3.060912151821243e-05],
  'media': 0.0005683466447057764,
  'win': 20},
 {'bSize': 20,
  'mAP_values': [0.002278097885797826,
   0.0009339937855790525,
   0.0002638309868445221,
   0.00012601376390416222,
   0.0],
  'media': 0.0007203872844251124,
  'win': 25},
 {'bSize': 20,
  'mAP_values': [0.002195457424690475,
   0.000861085725137292,
   0.00033403626518733624,
   8.560747006099881e-05,
   8.276005803541643e-06],
  'media': 0.0006968925781759287,
  'win': 30},
 {'bSize': 20,
  'mAP_values': [0.002104760958053995,
   0.0008382204643160299,
   0.0002634505305008862,
   4.867430570264462e-05,
   1.9841269841269845e-05],
  'media': 0.000654989505682965,
  'win': 35},
 {'bSize': 20,
  'mAP_values': [0.001745739278153592,
   0.0005393399712142924,
   0.00021457320121877902,
   4.686285131016999e-05,
   0.0],
  'media': 0.0005093030603793667,
  'win': 40}

## 7 tentativa - 

Modelo adaptado do **Lucas Valentim**

*   Sem os **dropout**
*   Função **Adadelta** para compilação
*   **BatchNormalization** antes da ultima camada Dense


In [0]:
architecture = 'architecture_7'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adadelta

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Dense(2, activation='sigmoid', kernel_regularizer=l2(1e-4)))

In [0]:
model.compile(Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_7/Janela20-Batch20/Resultado_Image
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_7/Janela20-Batch20/Resultado_Image/IOU4
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_7/Janela20-Batch20/Resultado_Image/IOU5
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_7/Janela20-Batch20/Resultado_Image/IOU6
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Result

[{'bSize': 20,
  'mAP_values': [0.0018470016215790665,
   0.0006871120035038309,
   0.00023886885302745167,
   5.7945908549807885e-05,
   1.2025012025012024e-05],
  'media': 0.0005685906797370339,
  'win': 20},
 {'bSize': 20,
  'mAP_values': [0.0025933170967966817,
   0.0010541516486969917,
   0.00026742001445256635,
   8.240455274535791e-05,
   0.0],
  'media': 0.0007994586625383194,
  'win': 25},
 {'bSize': 20,
  'mAP_values': [0.0026549326652287306,
   0.0010725580860638446,
   0.00043599479825360464,
   0.00011614641432666612,
   1.0180771555897945e-05],
  'media': 0.0008579625470857488,
  'win': 30},
 {'bSize': 20,
  'mAP_values': [0.0024506128033860235,
   0.0009433988061202194,
   0.000272917943352059,
   5.0692554002313366e-05,
   1.658374792703151e-05],
  'media': 0.0007468411709575294,
  'win': 35},
 {'bSize': 20,
  'mAP_values': [0.001880408828042434,
   0.0006946445438681924,
   0.00021512319689873623,
   3.714805874128897e-05,
   0.0],
  'media': 0.0005654649255101303,
  '

## 8 tentativa - 

Modelo adaptado do **Lucas Valentim**

*   Função **Adadelta** para compilação


In [0]:
architecture = 'architecture_8'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adadelta

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid', kernel_regularizer=l2(1e-4)))

In [0]:
model.compile(Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_8/Janela20-Batch20/Resultado_Image
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_8/Janela20-Batch20/Resultado_Image/IOU4
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_8/Janela20-Batch20/Resultado_Image/IOU5
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_8/Janela20-Batch20/Resultado_Image/IOU6
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Result

## 9 tentativa - 

Modelo adaptado do**Lucas Valentim**

*   Função **SGD** para compilação


In [0]:
architecture = 'architecture_9'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid', kernel_regularizer=l2(1e-4)))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.compile(SGD(lr=0.03, momentum=0.0, decay=0.0, nesterov=False), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 7
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run 

[{'bSize': 20,
  'mAP_values': [0.002138535398304329,
   0.0008888085901145731,
   0.00029507275372507667,
   7.162430862196276e-05,
   1.5782828282828283e-05],
  'media': 0.0006819647758097541,
  'win': 20},
 {'bSize': 20,
  'mAP_values': [0.0026074861049887725,
   0.0010226071705567708,
   0.0003030829883506334,
   8.765112948001767e-05,
   0.0],
  'media': 0.0008041654786752389,
  'win': 25},
 {'bSize': 20,
  'mAP_values': [0.002792890815271632,
   0.0010473063928160898,
   0.0003967595419071361,
   0.00010685198809455759,
   1.4809367691066378e-05],
  'media': 0.0008717236211560963,
  'win': 30},
 {'bSize': 20,
  'mAP_values': [0.0025105439533794865,
   0.0010222677781586604,
   0.000320657337240577,
   6.674718302425001e-05,
   1.7636684303350972e-05],
  'media': 0.0007875705872212649,
  'win': 35},
 {'bSize': 20,
  'mAP_values': [0.0020652936190377847,
   0.0007148821590820013,
   0.0002574294161618277,
   5.1762133273010077e-05,
   0.0],
  'media': 0.0006178734655109247,
  'win'

## 10 tentativa - 

Modelo adaptado do **Lucas Valentim**

*   Função **adam** para compilação
*   Adicionado mais 2 camadas convolutivas

In [0]:
architecture = 'architecture_10'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(96, (3, 3),activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(96, (3, 3),activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid', kernel_regularizer=l2(1e-4)))

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_10/Janela20-Batch20/Resultado_Image/IOU6
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_10/Janela20-Batch20/Resultado_Image/IOU7
IoU 7
Run image 12.tif
Run image 16.

[{'bSize': 20,
  'mAP_values': [0.0015257132252248133,
   0.0005814381151774256,
   0.00015587454991982337,
   3.8643704449593665e-05,
   0.0],
  'media': 0.0004603339189543313,
  'win': 20},
 {'bSize': 20,
  'mAP_values': [0.0017853782269231396,
   0.0006964310857546173,
   0.00018553690862320923,
   7.01072973587312e-05,
   0.0],
  'media': 0.0005474907037319395,
  'win': 25},
 {'bSize': 20,
  'mAP_values': [0.0019704229464478735,
   0.0007690862611127296,
   0.0002801612307022685,
   4.330475442106362e-05,
   6.899108715901014e-06],
  'media': 0.0006139748602799673,
  'win': 30},
 {'bSize': 20,
  'mAP_values': [0.0018684021694616527,
   0.0007105920188216131,
   0.00024721453092944667,
   4.088227263736284e-05,
   1.949317738791423e-05],
  'media': 0.0005773168338475979,
  'win': 35},
 {'bSize': 20,
  'mAP_values': [0.0015059715857051275,
   0.0005135932924514247,
   0.00021821317099578194,
   3.6429872495446266e-05,
   0.0],
  'media': 0.0004548415843295561,
  'win': 40}]

## 11 tentativa - 

Modelo adaptado do **Lucas Valentim**

*   Função **SGD** para compilação
*   Adicionado mais 2 camadas convolutivas

In [0]:
architecture = 'architecture_11'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4), 
                 input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(96, (3, 3),activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(96, (3, 3),activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (1,1), kernel_regularizer=l2(1e-4)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid', kernel_regularizer=l2(1e-4)))

In [0]:
model.compile(SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_11/Janela20-Batch20/Resultado_Image
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_11/Janela20-Batch20/Resultado_Image/IOU4
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_11/Janela20-Batch20/Resultado_Image/IOU5
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/architecture_11/Janela20-Batch20/Resultado_Image/IOU6
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resu

## 12 tentativa - 

Modelo adaptado do **Lucas Valentim**

*   Sem utilização das funções de regularização
*   Função **SGD** para compilação
*   Adicionado mais 2 camadas convolutivas

In [0]:
architecture = 'architecture_12'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu',input_shape=(None, None, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(96, (3, 3),activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(96, (3, 3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (1,1)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

In [0]:
model.compile(SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
              loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

## 13 tentativa - 

Modelo adaptado do **Lucas Valentim**

*   Utilizado outra função de normalização, diferente da utilizada por ele.

In [0]:
architecture = 'architecture_13'

### Modelo

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.regularizers import l2

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu',input_shape=(None, None, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, (7, 7),activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, (1,1)))
model.add(GlobalMaxPooling2D())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Treino


In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [0]:
ler_images_teste(architecture, model)

## Mini VGG - Base 2

In [0]:
architecture = 'miniVGG_base2_colorida'

### Modelo

In [0]:
def build_MiniVGG(width, height, depth, classes):
  # initialize the model
  model = Sequential()
  
  inputShape = (height, width, depth)# first CONV => RELU => CONV => RELU => POOL layer set
  model.add(Conv2D(32, (3, 3), padding="same",
  input_shape=inputShape))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=2))
  model.add(Conv2D(32, (3, 3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=2))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  # second CONV => RELU => CONV => RELU => POOL layer set
  model.add(Conv2D(64, (3, 3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=2))
  model.add(Conv2D(64, (3, 3), padding="same"))
  model.add(Activation("relu"))
  model.add(BatchNormalization(axis=2))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  # first (and only) set of FC => RELU layers
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation("relu"))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  
  # softmax classifier
  model.add(Dense(classes))
  model.add(Activation("softmax"))
  # return the constructed network architecture
  return model

In [0]:
model = build_MiniVGG(width=120, height=120, depth=3, classes=2)

opt = SGD(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

### Treino

In [0]:
model.load_weights('drive/My Drive/Resultado/Colorido/weights_' + architecture + '.hdf5')

### Teste

In [23]:
ler_images_teste(architecture, model)

bSize 20 -  win 20
Criou-se a pasta drive/My Drive/Resultado/Colorido/miniVGG_base2_colorida/Janela20-Batch20/Resultado_Image
Criou-se a pasta drive/My Drive/Resultado/Colorido/miniVGG_base2_colorida/Janela20-Batch20/Resultado_Image/IOU4
IoU 4
Run image 12.tif


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/miniVGG_base2_colorida/Janela20-Batch20/Resultado_Image/IOU5
IoU 5
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive/My Drive/Resultado/Colorido/miniVGG_base2_colorida/Janela20-Batch20/Resultado_Image/IOU6
IoU 6
Run image 12.tif
Run image 16.tif
Run image 24.tif
Run image 30.tif
Run image 32.tif
Run image 34.tif
Run image 48.tif
Run image 5.tif
Run image 56.tif
Run image 57.tif
Run image 58.tif
Run image 65.tif
Run image 69.tif
Run image 7.tif
Run image 73.tif
Criou-se a pasta drive

[{'bSize': 20,
  'mAP_values': [0.006133486977587758,
   0.0020851505533958175,
   0.0006624873074129529,
   0.00024654771754064226,
   0.0],
  'media': 0.0018255345111874341,
  'win': 20},
 {'bSize': 20,
  'mAP_values': [0.008077505708730188,
   0.004750230813545918,
   0.0014778525213390028,
   0.0008137278560853879,
   0.0],
  'media': 0.0030238633799400997,
  'win': 25},
 {'bSize': 20,
  'mAP_values': [0.007125236947827134,
   0.0032938977919920336,
   0.0012859089816526948,
   0.00042129115844324427,
   5.574136008918618e-05],
  'media': 0.0024364152480008585,
  'win': 30},
 {'bSize': 20,
  'mAP_values': [0.00655039577894102,
   0.004200308884109829,
   0.0011713505525218274,
   0.00037233131339107464,
   0.00017094017094017094],
  'media': 0.0024930653399807844,
  'win': 35},
 {'bSize': 20,
  'mAP_values': [0.003924569458223466,
   0.001298678589832048,
   0.0007822687292852674,
   0.0,
   0.0],
  'media': 0.0012011033554681565,
  'win': 40}]